In [ ]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/News-Recommendation')
sys.path.append('/home/peitian_zhang/Codes/News-Recommendation')

import torch
from utils.utils import evaluate,train,prepare,load_hparams
from models.NPA import NPAModel

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [ ]:
hparams = {
    'name':'npa',
    'epochs':10,
    'scale':'demo',
    'batch_size':100,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,mean_mrr,ndcg@5,ndcg@10',
    'attrs': ['title'],
    'k':-1,
    'save_step':None,
    'save_each_epoch':False,
    'train_embedding':False,
}

In [ ]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [ ]:
npaModel = NPAModel(vocab=vocab,hparams=hparams,uid2idx=loader_train.dataset.uid2index).to(device)

In [ ]:
train(npaModel, hparams, loader_train, loader_test, loader_validate)